### 1) 가짜 학생 점수 엑셀 생성

**문제**

Faker를 이용해 학생 이름 10명을 생성하고,NumPy로 국어·영어·수학 점수(0~100)를 만들어 엑셀 파일 `students.xlsx`에 저장하세요.

**조건**

- 시트명: `Scores`
- 컬럼: 이름, 국어, 영어, 수학

**힌트**

- Faker: `fake.name()`
- NumPy: `np.random.randint(0, 101, size=(10, 3))`
- openpyxl: `ws.append()`

In [3]:
import numpy as np
from pathlib import Path
from faker import Faker
from openpyxl import Workbook

np.random.seed(7)
N = 10
faker = Faker("ko_KR")
faker.seed_instance(7)
# print(faker.name())
# print(faker.email())
# print(faker.job())
names = [faker.name() for _ in range(N)]
emails = [faker.email() for _ in range(N)]
scores = np.random.randint(50, 101, size=(N, 3))  # 50~100점 사이의 정수 난수 생성  
print(names)

total_scores = scores.sum(axis=1) # 각 학생의 총점 계산
avg_scores = scores.mean(axis=1)  # 각 학생의 평균 계산
#print(total_scores)
#print(avg_scores)
wb = Workbook()
ws = wb.active
ws.title = "학생성적"
headers = ["이름", "이메일", "국어", "영어", "수학", "총점", "평균"]
ws.append(headers)
for idx in range(N):
    row = [
        names[idx],
        emails[idx],
        scores[idx, 0],
        scores[idx, 1],
        scores[idx, 2],
        total_scores[idx],
        round(avg_scores[idx], 2)
    ]
    ws.append(row)  
#file_path = Path(__file__).parent / "student_scores.xlsx"    #이건 일반 .py에서는 이렇게 쓰면 된다.
file_path = Path.cwd() / "student_scores.xlsx"  #이건 Jupyter Notebook에서는 이렇게 써야 한다.b.save(file_path)

['김재호', '박성호', '박광수', '김동현', '류민석', '강진호', '이현지', '고성민', '김성수', '김은경']


### 3) 평균 70점 이상 학생만 필터링

**문제**

학생 평균이 **70점 이상인 학생만** 새로운 시트 `Pass`에 저장하세요.

**힌트**

- 불리언 마스크: `avg >= 70`
- 배열 인덱싱: `data[mask]`